# String (Advanced)

### [Word Ladder](https://leetcode.com/problems/word-ladder/)

A transformation sequence from word beginWord to word endWord using a dictionary wordList is a sequence of words beginWord -> s1 -> s2 -> ... -> sk such that:

Every adjacent pair of words differs by a single letter.
Every si for 1 <= i <= k is in wordList. Note that beginWord does not need to be in wordList.
sk == endWord
Given two words, beginWord and endWord, and a dictionary wordList, return the number of words in the shortest transformation sequence from beginWord to endWord, or 0 if no such sequence exists.



In [ ]:
from collections import defaultdict
class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        """
        :type beginWord: str
        :type endWord: str
        :type wordList: List[str]
        :rtype: int
        """

        if endWord not in wordList or not endWord or not beginWord or not wordList:
            return 0

        # Since all words are of same length.
        L = len(beginWord)

        # Dictionary to hold combination of words that can be formed,
        # from any given word. By changing one letter at a time.
        all_combo_dict = defaultdict(list)
        for word in wordList:
            for i in range(L):
                # Key is the generic word
                # Value is a list of words which have the same intermediate generic word.
                all_combo_dict[word[:i] + "*" + word[i+1:]].append(word)


        # Queue for BFS
        queue = collections.deque([(beginWord, 1)])
        # Visited to make sure we don't repeat processing same word.
        visited = {beginWord: True}
        while queue:
            current_word, level = queue.popleft()
            for i in range(L):
                # Intermediate words for current word
                intermediate_word = current_word[:i] + "*" + current_word[i+1:]

                # Next states are all the words which share the same intermediate state.
                for word in all_combo_dict[intermediate_word]:
                    # If at any point if we find what we are looking for
                    # i.e. the end word - we can return with the answer.
                    if word == endWord:
                        return level + 1
                    # Otherwise, add it to the BFS Queue. Also mark it visited
                    if word not in visited:
                        visited[word] = True
                        queue.append((word, level + 1))
                all_combo_dict[intermediate_word] = []
        return 0

In [ ]:
from collections import defaultdict
class Solution(object):
    def __init__(self):
        self.length = 0
        # Dictionary to hold combination of words that can be formed,
        # from any given word. By changing one letter at a time.
        self.all_combo_dict = defaultdict(list)

    def visitWordNode(self, queue, visited, others_visited):
        queue_size = len(queue)
        for _ in range(queue_size):
            current_word = queue.popleft()
            for i in range(self.length):
                # Intermediate words for current word
                intermediate_word = current_word[:i] + "*" + current_word[i+1:]

                # Next states are all the words which share the same intermediate state.
                for word in self.all_combo_dict[intermediate_word]:
                    # If the intermediate state/word has already been visited from the
                    # other parallel traversal this means we have found the answer.
                    if word in others_visited:
                        return visited[current_word] + others_visited[word]
                    if word not in visited:
                        # Save the level as the value of the dictionary, to save number of hops.
                        visited[word] = visited[current_word] + 1
                        queue.append(word)
                        
        return None

    def ladderLength(self, beginWord, endWord, wordList):
        """
        :type beginWord: str
        :type endWord: str
        :type wordList: List[str]
        :rtype: int
        """
        if endWord not in wordList or not endWord or not beginWord or not wordList:
            return 0

        # Since all words are of same length.
        self.length = len(beginWord)

        for word in wordList:
            for i in range(self.length):
                # Key is the generic word
                # Value is a list of words which have the same intermediate generic word.
                self.all_combo_dict[word[:i] + "*" + word[i+1:]].append(word)

        # Queues for birdirectional BFS
        queue_begin = collections.deque([beginWord]) # BFS starting from beginWord
        queue_end = collections.deque([endWord]) # BFS starting from endWord

        # Visited to make sure we don't repeat processing same word
        visited_begin = {beginWord : 1}
        visited_end = {endWord : 1}
        ans = None

        # We do a birdirectional search starting one pointer from begin
        # word and one pointer from end word. Hopping one by one.
        while queue_begin and queue_end:
            
            # Progress forward one step from the shorter queue
            if len(queue_begin) <= len(queue_end):
                ans = self.visitWordNode(queue_begin, visited_begin, visited_end)
            else:
                ans = self.visitWordNode(queue_end, visited_end, visited_begin)
            if ans:
                return ans

        return 0

### [Alien Dictionary](https://leetcode.com/problems/alien-dictionary/)

There is a new alien language that uses the English alphabet. However, the order among the letters is unknown to you.

You are given a list of strings words from the alien language's dictionary, where the strings in words are sorted lexicographically by the rules of this new language.

Return a string of the unique letters in the new alien language sorted in lexicographically increasing order by the new language's rules. If there is no solution, return "". If there are multiple solutions, return any of them.

A string s is lexicographically smaller than a string t if at the first letter where they differ, the letter in s comes before the letter in t in the alien language. If the first min(s.length, t.length) letters are the same, then s is smaller if and only if s.length < t.length.



In [ ]:

from collections import defaultdict, Counter, deque

def alienOrder(self, words: List[str]) -> str:
    
    # Step 0: create data structures + the in_degree of each unique letter to 0.
    adj_list = defaultdict(set)
    in_degree = Counter({c : 0 for word in words for c in word})
            
    # Step 1: We need to populate adj_list and in_degree.
    # For each pair of adjacent words...
    for first_word, second_word in zip(words, words[1:]):
        for c, d in zip(first_word, second_word):
            if c != d:
                if d not in adj_list[c]:
                    adj_list[c].add(d)
                    in_degree[d] += 1
                break
        else: # Check that second word isn't a prefix of first word.
            if len(second_word) < len(first_word): return ""
    
    # Step 2: We need to repeatedly pick off nodes with an indegree of 0.
    output = []
    queue = deque([c for c in in_degree if in_degree[c] == 0])
    while queue:
        c = queue.popleft()
        output.append(c)
        for d in adj_list[c]:
            in_degree[d] -= 1
            if in_degree[d] == 0:
                queue.append(d)
                
    # If not all letters are in output, that means there was a cycle and so
    # no valid ordering. Return "" as per the problem description.
    if len(output) < len(in_degree):
        return ""
    # Otherwise, convert the ordering we found into a string and return it.
    return "".join(output)

In [ ]:

def alienOrder(self, words: List[str]) -> str:

    # Step 0: Put all unique letters into the adj list.
    reverse_adj_list = {c : [] for word in words for c in word}

    # Step 1: Find all edges and put them in reverse_adj_list.
    for first_word, second_word in zip(words, words[1:]):
        for c, d in zip(first_word, second_word):
            if c != d: 
                reverse_adj_list[d].append(c)
                break
        else: # Check that second word isn't a prefix of first word.
            if len(second_word) < len(first_word): 
                return ""

    # Step 2: Depth-first search.
    seen = {} # False = grey, True = black.
    output = []
    def visit(node):  # Return True iff there are no cycles.
        if node in seen:
            return seen[node] # If this node was grey (False), a cycle was detected.
        seen[node] = False # Mark node as grey.
        for next_node in reverse_adj_list[node]:
            result = visit(next_node)
            if not result: 
                return False # Cycle was detected lower down.
        seen[node] = True # Mark node as black.
        output.append(node)
        return True

    if not all(visit(node) for node in reverse_adj_list):
        return ""

    return "".join(output)


### longest common prefix
Given a array of N strings, find the longest common prefix among all strings present in the array.

In [1]:

# A python3 Program to find the longest 
# common prefix

# A Utility Function to find the common 
# prefix between strings- str1 and str2
def commonPrefixUtil(str1, str2):

    result = "";
    n1 = len(str1)
    n2 = len(str2)

    # Compare str1 and str2
    i = 0
    j = 0
    while i <= n1 - 1 and j <= n2 - 1:
    
        if (str1[i] != str2[j]):
            break
            
        result += str1[i]
        i += 1
        j += 1

    return (result)

# A Function that returns the longest 
# common prefix from the array of strings
def commonPrefix (arr, n):

    prefix = arr[0]

    for i in range (1, n):
        prefix = commonPrefixUtil(prefix, arr[i])

    return (prefix)

In [2]:
# Driver Code
if __name__ =="__main__":

    arr = ["geeksforgeeks", "geeks",
                    "geek", "geezer"]
    n = len(arr)

    ans = commonPrefix(arr, n)

    if (len(ans)):
        print ("The longest common prefix is -",
                ans);
    else:
        print("There is no common prefix")

# This code is contributed by ita_c



The longest common prefix is - gee


### Equal point
Given a string S of opening and closing brackets '(' and ')' only. The task is to find an equal point. An equal point is an index such that the number of closing brackets on right from that point must be equal to number of opening brackets before that point.

In [3]:

# Method to find an equal index
def findIndex(str):
    l = len(str)
    open = [0] * (l + 1)
    close = [0] * (l + 1)
    index = -1
    
    open[0] = 0
    close[l] = 0
    if (str[0]=='('):
        open[1] = 1
    if (str[l - 1] == ')'):
        close[l - 1] = 1
    
    # Store the number of
    # opening brackets
    # at each index
    for i in range(1, l):
        if (str[i] == '('):
            open[i + 1] = open[i] + 1
        else:
            open[i + 1] = open[i]
    
    # Store the number
    # of closing brackets
    # at each index
    for i in range(l - 2, -1, -1):
        if ( str[i] == ')'):
            close[i] = close[i + 1] + 1
        else:
            close[i] = close[i + 1]
    
    # check if there is no 
    # opening or closing brackets
    if (open[l] == 0):
        return len
    if (close[0] == 0):
        return 0
    
    # check if there is any 
    # index at which both
    # brackets are equal
    for i in range(l + 1):
        if (open[i] == close[i]):
            index = i
    
    return index
    
# Driver Code
str = "(()))(()()())))"
print(findIndex(str))

# This code is contributed 
# by ChitraNayal



9


### Minimum number of times A has to be repeated such that B is a Substring of it

Given two strings A and B. Find minimum number of times A has to be repeated such that B is a Substring of it. If B can never be a substring then return -1.

In [ ]:
class Solution:
    # Function to check if a number  
    # is a substring of other or not 
    def issubstring(self, str2, rep):
        if str2 in rep:
            return True  # str2 is a Substring of rep
        return False     # not a substring
    
    def minRepeats(self, A, B):
        ans = 1
        S = A
        
        while len(S)<len(B):
            S += A 
            ans += 1 
            
        if self.issubstring(B,S):
            return ans 
        elif self.issubstring(B,S+A):
            return ans+1 
        else:
            return -1

4. We are given a binary string containing 1’s and 0’s. Find the maximum length of consecutive 1’s in it.

In [7]:
# Function to find Maximum length of consecutive 1's in a binary string

def maxConsecutive1(input):
	# input.split('0') --> splits all sub-strings of consecutive 1's
	# separated by 0's, output will be like ['11','1111','1','1','111']
	# map(len,input.split('0')) --> map function maps len function on each
	# sub-string of consecutive 1's
	# max() returns maximum element from a list
    
	print(max(map(len,input.split('0'))))

# Driver program
if __name__ == "__main__":
	input = '11000111101010111'
	maxConsecutive1(input)



4


In [12]:
input = '11000111101010111'
input.split('0')

['11', '', '', '1111', '1', '1', '111']

In [11]:
input = '11000111101010111'
list(map(len,input.split('0')))

[2, 0, 0, 4, 1, 1, 3]

In [21]:
map(len,(100,10,1,0))

### Print all the common characters in lexicographical order

Given two strings, print all the common characters in lexicographical order. If there are no common letters, print -1. All letters are lower case.

In [ ]:
# Function to print common characters of two Strings
# in alphabetical order
from collections import Counter

def common(str1,str2):
	
	# convert both strings into counter dictionary
	dict1 = Counter(str1)
	dict2 = Counter(str2)

	# take intersection of these dictionaries
	commonDict = dict1 & dict2

	if len(commonDict) == 0:
		print (-1)
		return

	# get a list of common elements
	commonChars = list(commonDict.elements())

	# sort list in ascending order to print resultant
	# string on alphabetical order
	commonChars = sorted(commonChars)

	# join characters without space to produce
	# resultant string
	print (''.join(commonChars))

# Driver program
if __name__ == "__main__":
	str1 = 'geeks'
	str2 = 'forgeeks'
	common(str1, str2)



### Make them Anagram
Given two strings in lowercase, the task is to make them Anagram. The only allowed operation is to remove a character from any string. Find minimum number of characters to be deleted to make both the strings anagram?
If two strings contains same data set in any order then strings are called Anagrams.

In [22]:
# Function remove minimum number of characters so that
# two strings become anagram
from collections import Counter
def removeChars(str1, str2):

	# make dictionaries from both strings
	dict1 = Counter(str1)
	dict2 = Counter(str2)

	# extract keys from dict1 and dict2
	keys1 = dict1.keys()
	keys2 = dict2.keys()

	# count number of keys in both lists of keys
	count1 = len(keys1)
	count2 = len(keys2)

	# convert list of keys in set to find common keys
	set1 = set(keys1)
	commonKeys = len(set1.intersection(keys2))

	if (commonKeys == 0):
		return count1 + count2
	else:
		return (max(count1, count2)-commonKeys)

# Driver program
if __name__ == "__main__":
	str1 ='bcadeh'
	str2 ='hea'
	print (removeChars(str1, str2))



3


### Groupby to remove consecutive dublicates

Group by method takes two input one is iterable (list,tuple,dictionary) and second is key function which calculates keys for each element present in iterable. It returns key and iterable of grouped items. If key function not specified or is None, key defaults to an identity function and returns the element unchanged. For example,

In [25]:
numbers = [1, 1, 1, 3, 3, 2, 2, 2, 1, 1]
import itertools
for (key,group) in itertools.groupby(numbers):
	print (key,list(group))



1 [1, 1, 1]
3 [3, 3]
2 [2, 2, 2]
1 [1, 1]


In [26]:
# function to remove all consecutive duplicates
# from the string in Python

from itertools import groupby
def removeAllConsecutive(input):
	
	# group all consecutive characters based on their
	# order in string and we are only concerned
	# about first character of each consecutive substring
	# in given string, so key value will work for us
	# and we will join these keys without space to
	# generate resultant string
	result = []
	for (key,group) in groupby(input):
		result.append(key)

	print (''.join(result))
	
# Driver program
if __name__ == "__main__":
	input = 'aaaaabbbbbb'
	removeAllConsecutive(input)



ab


### Mirror Characters

Given a string and a number N, we need to mirror the characters from the N-th position up to the length of the string in alphabetical order. In mirror operation, we change ‘a’ to ‘z’, ‘b’ to ‘y’, and so on.


In [28]:
original = 'abcdefghijklmnopqrstuvwxyz'
reverse = 'zyxwvutsrqponmlkjihgfedcba'
dictChars = dict(zip(original,reverse))
dictChars

{'a': 'z',
 'b': 'y',
 'c': 'x',
 'd': 'w',
 'e': 'v',
 'f': 'u',
 'g': 't',
 'h': 's',
 'i': 'r',
 'j': 'q',
 'k': 'p',
 'l': 'o',
 'm': 'n',
 'n': 'm',
 'o': 'l',
 'p': 'k',
 'q': 'j',
 'r': 'i',
 's': 'h',
 't': 'g',
 'u': 'f',
 'v': 'e',
 'w': 'd',
 'x': 'c',
 'y': 'b',
 'z': 'a'}

In [27]:
# function to mirror characters of a string

def mirrorChars(input,k):

	# create dictionary
	original = 'abcdefghijklmnopqrstuvwxyz'
	reverse = 'zyxwvutsrqponmlkjihgfedcba'
	dictChars = dict(zip(original,reverse))

	# separate out string after length k to change
	# characters in mirror
	prefix = input[0:k-1]
	suffix = input[k-1:]
	mirror = ''

	# change into mirror
	for i in range(0,len(suffix)):
		mirror = mirror + dictChars[suffix[i]]

	# concat prefix and mirrored part
	print (prefix+mirror)
		
# Driver program
if __name__ == "__main__":
	input = 'paradox'
	k = 3
	mirrorChars(input,k)



paizwlc


### Given a string S, c1 and c2. Replace character c1 with c2 and c2 with c1.

In [29]:
# Function to replace a character c1 with c2
# and c2 with c1 in a string S

def replaceChars(input,c1,c2):

	# create lambda to replace c1 with c2, c2
	# with c1 and other will remain same
	# expression will be like "lambda x:
	# x if (x!=c1 and x!=c2) else c1 if (x==c2) else c2"
	# and map it onto each character of string
    
	newChars = map(lambda x: x if (x!=c1 and x!=c2) else \
				c1 if (x==c2) else c2,input)

	# now join each character without space
	# to print resultant string
	print (''.join(newChars))

# Driver program
if __name__ == "__main__":
	input = 'grrksfoegrrks'
	c1 = 'e'
	c2 = 'r'
	replaceChars(input,c1,c2)



geeksforgeeks
